# Run FunSearch on Bin Packing
Five steps:
1. Implement 'LLM' interface.
2. Implement a 'SandBox' interface.
3. Prepare a 'specification'.
4. Prepare a dataset.
5. Start FunSearch.

## Preparation: download the project file from github. And update system path.

In [1]:
'''import shutil

folder_path = "/content/TSP_funsearch"
shutil.rmtree(folder_path)  # delete existing tsp if needed'''

'import shutil\n\nfolder_path = "/content/TSP_funsearch"\nshutil.rmtree(folder_path)  # delete existing tsp if needed'

In [2]:
!git clone https://github.com/Perkzi/TSP_funsearch.git

import sys

sys.path.append('/content/TSP_funsearch/')
print(sys.path)

fatal: destination path 'TSP_funsearch' already exists and is not an empty directory.
['/content', '/env/python', '/usr/lib/python311.zip', '/usr/lib/python3.11', '/usr/lib/python3.11/lib-dynload', '', '/usr/local/lib/python3.11/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.11/dist-packages/IPython/extensions', '/root/.ipython', '/content/TSP_funsearch/']


## 1. Implement LLM interface
Set the API's IP address according to your API provider (See line 65 in the following code).
```python
conn = http.client.HTTPSConnection("api.chatanywhere.com.cn")
```
You should prepare a 'key' for the LLM API. And fill them in the header (See line 76-80 in the following code).
```python
headers = {
    'Authorization': 'Bearer [put your key here, the key may start with "sk-..."]',
    'User-Agent': 'Apifox/1.0.0 (https://apifox.com)',
    'Content-Type': 'application/json'
}
```

In [3]:
import time
import json
import multiprocessing
from typing import Collection, Any
import http.client
import traceback
from implementation import sampler



def _trim_preface_of_body(sample: str) -> str:
    """Trim the redundant descriptions/symbols/'def' declaration before the function body.
    Please see my comments in sampler.LLM (in sampler.py).
    Since the LLM used in this file is not a pure code completion LLM, this trim function is required.

    -Example sample (function & description generated by LLM):
    -------------------------------------
    This is the optimized function ...
    def priority_v2(...) -> ...:
        return ...
    This function aims to ...
    -------------------------------------
    -This function removes the description above the function's signature, and the function's signature.
    -The indent of the code is preserved.
    -Return of this function:
    -------------------------------------
        return ...
    This function aims to ...
    -------------------------------------
    去掉def之前和自己的行
    """
    lines = sample.splitlines()
    func_body_lineno = 0
    find_def_declaration = False
    for lineno, line in enumerate(lines):
        # find the first 'def' statement in the given code
        if line[:3] == 'def':
            func_body_lineno = lineno
            find_def_declaration = True
            break
    if find_def_declaration:
        code = ''
        for line in lines[func_body_lineno + 1:]:
            code += line + '\n'
        return code
    return sample

from openai import OpenAI
import time
from typing import Collection
#from TSP_algorithms.additional_prompts import base_prompt as ap

# 1.choose a different additional_prompts
# 'base_prompt' 'step_by_step_prompts'  'multi_path_analysis_prompts'
# 'heuristic_meta_prompts' 'multi_phase_prompts' 'self_analysis_prompts'
from TSP_algorithms.additional_prompts2 import step_by_step_prompts as ap

class LLMAPI:
    def __init__(self, samples_per_prompt: int, trim=True):
        API_KEY = "" # 设置 OpenAI API 密钥
        BASE_URL = 'https://api.bltcy.ai/v1'
        self.client = OpenAI(api_key=API_KEY, base_url=BASE_URL)
        self._samples_per_prompt = samples_per_prompt
        self._trim = trim
        if ap:
            self._additional_prompt = ap
        else:
            self._additional_prompt = (
                "Complete a different and more complex Python function. "
                "Be creative and you can insert multiple if-else and for-loop in the code logic. "
                "Only output the Python code, no descriptions."
            )
        if type(self._additional_prompt) is not list:
          self._additional_prompt = [self._additional_prompt]

    def draw_samples(self, prompt: str) -> Collection[str]:
        """Returns multiple predicted continuations of `prompt`."""
        #print("draw-samples")
        return [self._draw_sample(prompt) for _ in range(self._samples_per_prompt)]

    def _draw_sample(self, content: str) -> str:
        max_retries = 3

        # whether have the pre prompt before main prompt
        if len(self._additional_prompt)<=1:
          prompt = '\n'.join([content, self._additional_prompt[0]])
        elif len(self._additional_prompt)==2:
          pre_prompt = '\n'.join([content, self._additional_prompt[0]])
          retries = 0
          pre_output = ''
          prompt = ''
          while retries < max_retries:
            try:
                #print("prompt")
                response = self.client.chat.completions.create(
                    model="gpt-3.5-turbo",
                    messages=[{"role": "user", "content": pre_prompt}],
                    max_tokens=512,
                    stream=False,
                )
                print("pre_response",response)
                pre_output = response.choices[0].message.content
                # 将 第二个prompt中的 <insert optimization analysis here> 替换为分析结果。
                prompt = '\n'.join([content, self._additional_prompt[1].replace("<insert optimization analysis here>", pre_output)])
                break
                #return output
            except Exception as e:
                traceback.print_exc()
                print(f"Error occurred: {e}")
                retries += 1
                time.sleep(2)
        else:
          raise RuntimeError("list of additional_prompt should have length <= 2")

        # the main prompt
        retries = 0
        output = ''
        while retries < max_retries:
            try:
                #print("prompt")
                response = self.client.chat.completions.create(
                    model="gpt-3.5-turbo",
                    messages=[{"role": "user", "content": prompt}],
                    max_tokens=512,
                    stream=False,
                )
                print("response",response)
                output = response.choices[0].message.content

                if self._trim:
                    output = _trim_preface_of_body(output)
                break
                #return output
            except Exception as e:
                traceback.print_exc()
                print(f"Error occurred: {e}")
                retries += 1
                time.sleep(2)
        if output == '':
            print("Failed")
            raise RuntimeError("Failed after multiple retries")
        return output

# test LLMAPI
'''
llm = LLMAPI(samples_per_prompt=4)
response = llm._draw_sample('hello')
print(response)
'''

"\nllm = LLMAPI(samples_per_prompt=4)\nresponse = llm._draw_sample('hello')\nprint(response)\n"

## 2. Implement a 'SandBox' interface

In [4]:
from implementation import evaluator
from implementation import evaluator_accelerate


class Sandbox(evaluator.Sandbox):
    """Sandbox for executing generated code. Implemented by RZ.

    RZ: Sandbox returns the 'score' of the program and:
    1) avoids the generated code to be harmful (accessing the internet, take up too much RAM).
    2) stops the execution of the code in time (avoid endless loop).
    """

    def __init__(self, verbose=False, numba_accelerate=True):
        """
        Args:
            verbose         : Print evaluate information.
            numba_accelerate: Use numba to accelerate the evaluation. It should be noted that not all numpy functions
                              support numba acceleration, such as np.piecewise().
        """
        self._verbose = verbose
        self._numba_accelerate = numba_accelerate

    def run(
            self,
            program: str,
            function_to_run: str,  # RZ: refers to the name of the function to run (e.g., 'evaluate')
            function_to_evolve: str,  # RZ: accelerate the code by decorating @numba.jit() on function_to_evolve.
            inputs: Any,  # refers to the dataset # self._inputs是数据集的dict，current_input是当前使用的数据集的key (名字str)
            test_input: str,  # refers to the current instance
            timeout_seconds: int,
            **kwargs  # RZ: add this
    ) -> tuple[Any, bool]:
        """Returns `function_to_run(test_input)` and whether execution succeeded.

        RZ: If the generated code (generated by LLM) is executed successfully,
        the output of this function is the score of a given program.
        RZ: PLEASE NOTE THAT this SandBox is only designed for bin-packing problem.
        返回 (function_to_run(test_input)运行结果，True)
        """
        dataset = inputs[test_input]
        try:
            # 当你有多个进程时，每个进程都会将它们的结果放入各自的 result_queue 中？
            result_queue = multiprocessing.Queue()
            # 创建一个新的进程 process，目标函数为 self._compile_and_run_function，并传递相应的参数。只并行了一个进程
            process = multiprocessing.Process(
                target=self._compile_and_run_function,
                args=(program, function_to_run, function_to_evolve, dataset, self._numba_accelerate, result_queue)
            )
            process.start()
            # 等待进程完成。如果进程在 timeout_seconds 时间内未完成，继续执行后续代码
            process.join(timeout=timeout_seconds)
            if process.is_alive():
                # 如果进程仍然在运行（超时未完成）
                # if the process is not finished in time, we consider the program illegal
                process.terminate()
                process.join()
                results = None, False
            else:
                # 如果进程已完成，从队列中获取结果。
                if not result_queue.empty():
                    results = result_queue.get_nowait()
                else:
                    results = None, False

            return results
        except:
            return None, False

    def _compile_and_run_function(self, program, function_to_run, function_to_evolve, dataset, numba_accelerate,
                                  result_queue):
        try:
            # optimize the code (decorate function_to_run with @numba.jit())
            # 加上 @numba.jit() 装饰器
            if numba_accelerate:
                program = evaluator_accelerate.add_numba_decorator(
                    program=program,
                    function_to_evolve=function_to_evolve
                )
            # compile the program, and maps the global func/var/class name to its address
            all_globals_namespace = {}
            # execute the program, map func/var/class to global namespace
            # exec 是一个内置的Python函数，用于动态执行Python代码。
            # program 是一个包含要执行代码的字符串。all_globals_namespace 是一个字典，用于保存执行代码期间创建的全局变量、函数和类
            exec(program, all_globals_namespace)
            # get the pointer of 'function_to_run'
            # 从 all_globals_namespace 中获取要运行的函数
            function_to_run = all_globals_namespace[function_to_run]
            # return the execution results
            results = function_to_run(dataset)
            # the results must be int or float
            if not isinstance(results, (int, float)):
                result_queue.put((None, False))
                return
            result_queue.put((results, True))
        except Exception:
            # if raise any exception, we assume the execution failed
            result_queue.put((None, False))

## 3. Prepare a 'specification'

In [5]:
# 2.choose a different initial algorithm
# 'TSP_algorithm_cheapest' 'TSP_algorithm_farthest' 'TSP_algorithm_nearest'
from TSP_algorithms import TSP_algorithm_cheapest as algo
specification = algo.specification
print(specification)



import numpy as np
import time

def tsp_evaluate(route: list[int], distances: np.ndarray) -> float:
    return (
        sum(distances[route[i], route[i + 1]] for i in range(len(route) - 1))
        + distances[route[-1], route[0]]
    )

@funsearch.evolve
def tsp_priority(
    current_path: list[int],          # 目前已形成的回路（首尾不同）
    unvisited: list[int],             # 仍待插入的城市
    distances: np.ndarray,            # 距离矩阵
) -> tuple[int, int]:
    """
    选择 (city_to_insert, insert_position)，使新增距离增量最小。
    insert_position 表示要把 city 插到 current_path 的哪个索引之前。
    """
    best_delta = float("inf")
    best_city, best_position = unvisited[0], 1

    path_len = len(current_path)
    for candidate_city in unvisited:
        # 枚举 current_path 上的每条边 (i -> j)
        for idx in range(path_len):
            i = current_path[idx]
            j = current_path[(idx + 1) % path_len]  # % 保证最后连回起点
            delta = (
                distances[i, candidate_city]
                + distances[candidate_ci

## 4. Prepare a dataset

In [6]:
!pip install tsplib95

In [7]:
#from TSP_data import bin_packing_utils
#bin_packing_or3 = {'OR3': bin_packing_utils.datasets['OR3']}

from TSP_data import TSP_DataLoading
TSP_datasets = {}

instances_AlgorithmDevelop = TSP_DataLoading.build_funsearch_dataset(TSP_DataLoading.tsplib_data1)
instances_PerformanceTesting = TSP_DataLoading.build_funsearch_dataset(TSP_DataLoading.tsplib_data2)
instances_GeneralizationTesting = TSP_DataLoading.build_funsearch_dataset(TSP_DataLoading.tsplib_data3)

for name, instance in instances_AlgorithmDevelop.items():
  TSP_datasets[name] = {'section1':instance}

for name, instance in instances_PerformanceTesting.items():
  TSP_datasets[name] = {'section1':instance}

for name, instance in instances_GeneralizationTesting.items():
  TSP_datasets[name] = {'section1':instance}

# 3.选一个数据集
# 'berlin52', 'eil76' 'kroA100' 'pr124' 'd198' 'lin318' 'pcb442'
name = 'berlin52'
dataset_to_eval = {name: TSP_datasets[name]}



## 5. Start FunSearch
Please note that in jupyter notebook the following code will fail. This is because juypter does not support multiprocessing. Colab backend supports multiprocessing.

In [8]:
from implementation import funsearch
from implementation import config
import dataclasses

# It should be noted that the if __name__ == '__main__' is required.
# Because the inner code uses multiprocess evaluation.
if __name__ == '__main__':
    class_config = config.ClassConfig(llm_class=LLMAPI, sandbox_class=Sandbox)

    # default setting
    config = config.Config(samples_per_prompt=4, evaluate_timeout_seconds=30)
    global_max_sample_num = 10  # if it is set to None, funsearch will execute an endless loop

    # test setting
    #config = config.Config(samples_per_prompt=1, evaluate_timeout_seconds=30,programs_database=config.ProgramsDatabaseConfig(num_islands=1))
    #global_max_sample_num = 2

    funsearch.main(
        specification=specification,
        inputs=dataset_to_eval,
        config=config,
        max_sample_nums=global_max_sample_num,
        class_config=class_config,
        log_dir='../logs/funsearch_llm_api'
    )


import numpy as np
import time
 [Function(name='tsp_evaluate', args='route: list[int], distances: np.ndarray', body='    return (\n        sum(distances[route[i], route[i + 1]] for i in range(len(route) - 1))\n        + distances[route[-1], route[0]]\n    )', return_type='float', docstring=None, score=None, global_sample_nums=None, sample_time=None, evaluate_time=None), Function(name='tsp_priority', args='current_path: list[int], unvisited: list[int], distances: np.ndarray', body='    best_delta = float("inf")\n    best_city, best_position = unvisited[0], 1\n\n    path_len = len(current_path)\n    for candidate_city in unvisited:\n        # 枚举 current_path 上的每条边 (i -> j)\n        for idx in range(path_len):\n            i = current_path[idx]\n            j = current_path[(idx + 1) % path_len]  # % 保证最后连回起点\n            delta = (\n                distances[i, candidate_city]\n                + distances[candidate_city, j]\n                - distances[i, j]\n            )\n            i

INFO:absl:Best score of island 0 increased to -9050.693986415012
INFO:absl:Best score of island 1 increased to -9050.693986415012
INFO:absl:Best score of island 2 increased to -9050.693986415012
INFO:absl:Best score of island 3 increased to -9050.693986415012
INFO:absl:Best score of island 4 increased to -9050.693986415012
INFO:absl:Best score of island 5 increased to -9050.693986415012
INFO:absl:Best score of island 6 increased to -9050.693986415012
INFO:absl:Best score of island 7 increased to -9050.693986415012
INFO:absl:Best score of island 8 increased to -9050.693986415012
INFO:absl:Best score of island 9 increased to -9050.693986415012


================= Evaluated Function =================
def tsp_priority(current_path: list[int], unvisited: list[int], distances: np.ndarray) -> tuple[int, int]:
    """
    选择 (city_to_insert, insert_position)，使新增距离增量最小。
    insert_position 表示要把 city 插到 current_path 的哪个索引之前。
    """
    best_delta = float("inf")
    best_city, best_position = unvisited[0], 1

    path_len = len(current_path)
    for candidate_city in unvisited:
        # 枚举 current_path 上的每条边 (i -> j)
        for idx in range(path_len):
            i = current_path[idx]
            j = current_path[(idx + 1) % path_len]  # % 保证最后连回起点
            delta = (
                distances[i, candidate_city]
                + distances[candidate_city, j]
                - distances[i, j]
            )
            if delta < best_delta:
                best_delta = delta
                best_city = candidate_city
                best_position = idx + 1          # 插到 j 前面

    return best_city, best_position
--------------------

INFO:httpx:HTTP Request: POST https://api.bltcy.ai/v1/chat/completions "HTTP/1.1 200 OK"


pre_response ChatCompletion(id='chatcmpl-BPWolYRxtycBpym2ET4ovhFErxFSh', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The current heuristic approach to solving the TSP problem utilizes a priority-based strategy to determine the next city to visit and where to insert it in the current path. The algorithm iterates over all unvisited cities and potential insertion points in the current path to find the combination that minimizes the increase in total distance.\n\nTo optimize the algorithm, we can break it down into several structured phases:\n\n1. Initialization:\n   - Initialize the current path with a starting city and an empty list of unvisited cities.\n   - Compute the distance matrix between all pairs of cities.\n\n2. Iterative Improvement:\n   - In each iteration, consider all unvisited cities as potential candidates for the next city to visit.\n   - For each candidate city, calculate the delta in total distance if the city is 

...

response ChatCompletion(id='chatcmpl-BPWq3K4UD2ixfOmbaBKIJtWq95NhM', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='```python\nimport numpy as np\nimport time\n\ndef tsp_priority_v2(current_path: list[int], unvisited: list[int], distances: np.ndarray) -> tuple[int, int]:\n    best_delta = float("inf")\n    best_city, best_position = unvisited[0], 1\n\n    path_len = len(current_path)\n    for candidate_city in unvisited:\n        for idx in range(path_len):\n            i = current_path[idx]\n            j = current_path[(idx + 1) % path_len]\n            delta = (\n                distances[i, candidate_city]\n                + distances[candidate_city, j]\n                - distances[i, j]\n            )\n            if delta < best_delta:\n                best_delta = delta\n                best_city = candidate_city\n                best_position = idx + 1\n\n    return best_city, best_position\n```', refusal=None, role='assista

INFO:httpx:HTTP Request: POST https://api.bltcy.ai/v1/chat/completions "HTTP/1.1 200 OK"


pre_response ChatCompletion(id='chatcmpl-BPWq5TzsGrD5ovJRkx1qKWwK1ieHt', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The current heuristic approach for solving the TSP problem, as seen in the given `tsp_priority_v0` function, utilizes a priority-based strategy to determine which city to insert next into the current path in order to minimize the increase in distance. However, the implementation lacks a clear structure and detailed explanation of the optimization logic.\n\nTo optimize the algorithm, we can first define clear phases such as initialization, iterative improvement, and termination. In the initialization phase, necessary data structures are set up (e.g., current_path, unvisited cities, distance matrix). Iterative improvement involves iteratively selecting the best city to insert into the path based on the priority criterion until all cities are visited. Termination occurs when all cities have been visited and the optima

INFO:httpx:HTTP Request: POST https://api.bltcy.ai/v1/chat/completions "HTTP/1.1 200 OK"


response ChatCompletion(id='chatcmpl-BPWqA8APWZPYb0yGiuXpHV35YIQ2e', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='```python\nimport numpy as np\n\ndef tsp_priority_v2(distances: np.ndarray) -> tuple[int, list[int]]:\n    def calculate_cost(path: list[int]) -> int:\n        cost = 0\n        for i in range(len(path)):\n            cost += distances[path[i - 1]][path[i]]\n        return cost\n\n    def find_best_insertion(candidate_city: int, current_path: list[int]) -> tuple[int, int]:\n        best_cost = float("inf")\n        best_position = 0\n\n        for idx in range(1, len(current_path)):\n            new_path = current_path[:idx] + [candidate_city] + current_path[idx:]\n            new_cost = calculate_cost(new_path)\n            if new_cost < best_cost:\n                best_cost = new_cost\n                best_position = idx\n\n        return best_cost, best_position\n\n    unvisited_cities = list(range(1, len(distances)